In [ ]:
from pathlib import Path

import pandas as pd
import pytorch_lightning as pl
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from src.data.components.Dataset import GenericDatasetTransformer
from src.data.text_processing import TextPreprocessor
from src.utils import defines

In [ ]:
train_df = pd.read_csv(Path(defines.INTERIM_DATA_DIR, "val.csv"))
text_processor = TextPreprocessor()
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [ ]:
train_df["text"] = text_processor.transform_series(train_df["text"])
train_df["text"] = train_df["text"].apply(lambda x: " ".join(x))

In [ ]:
train_df["text_encoded"] = train_df["text"].apply(
    lambda x: tokenizer.encode_plus(
        x,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",
    )
)

In [ ]:
print(model)

In [ ]:
model

In [ ]:
pl.seed_everything(32)

In [ ]:
model.classifier

In [ ]:
text = train_df["text_encoded"][0]["input_ids"]

with torch.no_grad():
    x = model.roberta(text)
    x = model.classifier(x.last_hidden_state)
    test2 = model(text)
    print(torch.eq(x, test2.logits))

In [ ]:
model.modules()

In [ ]:
test = model(train_df["text_encoded"][0]["input_ids"])

In [ ]:
test.logits

In [ ]:
torch.nn.Sequence(model.base_model)
list(model.base_model.modules())

In [ ]:
train_df["text_encoded"][0]["input_ids"]

In [ ]:
logits_a = torch.rand((16, 2))
logits_b = torch.rand((16, 4))
logits_c = torch.rand((16, 11))

In [ ]:
logits_a